In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
%matplotlib inline
import numpy as np

from skimage.io import imread
from skimage.feature import canny

In [ ]:
# find edges in image
img = imread('house.tif')
edges_img = canny(img,  sigma=1, low_threshold=55, high_threshold=150)

plt.figure(figsize=(14,6))
plt.subplot(121)
plt.imshow(img,cmap = 'gray')
plt.title('Original image', fontsize=20)
plt.axis('off')

plt.subplot(122)
plt.imshow(edges_img,cmap = 'gray')
plt.title('Edge image', fontsize=20)
plt.axis('off')

plt.show()

## Hough transformation with skimage

In [ ]:
from skimage.transform import hough_line, hough_line_peaks

# create a vector of angles with steps of of 0.5 degree.
tested_angles = np.linspace(-np.pi/2, np.pi/2, 360, endpoint=False)

# Classic straight-line Hough transform
h, theta, r = hough_line(edges_img, theta=tested_angles)
# theta and r are vectors containing the Hough space bin coordinates

In [ ]:
# plotting the hough space
plt.figure(figsize=(14, 7))

# create correct axis labels
angle_step = 0.5 * np.diff(theta).mean()
r_step = 0.5 * np.diff(r).mean()
bounds = [np.rad2deg(theta[0] - angle_step),
          np.rad2deg(theta[-1] + angle_step),
          r[-1] + r_step, r[0] - r_step]

#plt.imshow(h, extent=bounds, cmap='gray', aspect=0.05)
plt.imshow(np.log(h+1), extent=bounds, cmap='gray', aspect=0.05)
plt.title('Hough space', size = 16)
plt.xlabel('Angles', size = 16)
plt.ylabel('Distance', size = 16)

plt.show()

In [ ]:
# find the 8 highest peaks in hough space
lines = zip(*hough_line_peaks(h, theta, r, num_peaks = 8))

# plot the detected lines
plt.figure(figsize=(7, 7))

plt.imshow(img, cmap='gray')

# for each line we compute the point in the first and last column
x_values = np.array((0, img.shape[1]))

# plot each line using the angle and distance
for _, angle, dist in lines:    # the first parameter is the line occupancy          
    y0, y1 = (dist - x_values * np.cos(angle)) / np.sin(angle)
    plt.plot(x_values, (y0, y1), '-r')

# we need to adapt the plot size to the image size    
plt.xlim(x_values)
plt.ylim((img.shape[0], 0))  # flipped, to agree with pixel coordinates

plt.axis('off')
plt.title('Detected lines')

plt.show()

## Hough transformation with opencv

In [ ]:
edges_img.dtype

In [ ]:
import cv2

# cv.HoughLines(image, rho resolution, theta resolution , threshold)   
lines = cv2.HoughLines(np.uint8(edges_img), 1, np.pi/360, 165)

# create image that will display the results in BGR
cdst = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    
if lines is not None:
    for i in range(0, len(lines)):
        rho = lines[i][0][0]
        theta = lines[i][0][1]
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a*rho
        y0 = b*rho
        x1 = int(x0 + 1000*(-b))
        y1 = int(y0 + 1000*(a))
        x2 = int(x0 - 1000*(-b))
        y2 = int(y0 - 1000*(a))
        #draw line into image
        cv2.line(cdst,(x1,y1),(x2,y2),(0,0,255),1)

        
plt.figure(figsize=(7,7))
plt.imshow(cdst,cmap = 'gray')
plt.title('Hough transform with openCV', fontsize=18)
plt.axis('off')

plt.show()